In [1]:
import ee
import geemap

In [2]:
ee.Initialize()

In [3]:
Map = geemap.Map()
roi = ee.FeatureCollection('projects/ee-qingqiu760/assets/jiangxi').filter(ee.Filter.eq('NAME', '于都县'))
Map.addLayer(roi, {}, '于都县')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# ee.Image 初步认识
## 查阅官方文档学习
## https://developers.google.com/earth-engine/apidocs/ee-image

In [4]:
# 哨兵2 去云函数
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
    qa.bitwiseAnd(cirrusBitMask).eq(0))

    # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask).divide(10000) \
    .select("B.*") \
    .copyProperties(image, ["system:time_start"])

In [5]:
sentinel = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
                .filterBounds(roi) \
                .filterDate('2024-01-01', '2024-12-31') \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 0.1)) \
                .map(maskS2clouds) \
                .median() \
                .clip(roi)

In [6]:
visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

In [7]:
Map.addLayer(sentinel, visualization, 'Sentinel')
Map

Map(bottom=56141.0, center=[25.71826065792773, 115.93872070312501], controls=(WidgetControl(options=['position…

## 单个影像中可以有许多波段
## 通过 **select** 函数可以将单个波段选取出来
## 通过 **addBands** 函数可以将影像作为波段加入另一个影像

In [8]:
blue = sentinel.select('B2').rename('blue')
green = sentinel.select('B3').rename('green')
red = sentinel.select('B4').rename('red')

In [9]:
rgb = ee.Image.cat([red, green, blue])    # 将 RGB 波段合并为一个图像
rgb = blue.addBands(green).addBands(red)  # 另一种合并方式
rgb.bandNames().getInfo()  # 获取波段名称

['blue', 'green', 'red']

# **影像的计算**

---
## geemap 中影像的计算有别与 numpy 的二维数组
## **不能**通过 加（+）、减（-）、乘（*）、除（/）等运算符计算

---
## ee.Image 有其独有的计算方式
1. 加：ee.Image.add()
2. 减：ee.Image.subtract()
3. 乘：ee.Image.multiply()
4. 除：ee.Image.divide()
## 需要对 ee.Image 对象使用，参数为需要运算的**数字**或**影像**

In [10]:
# ee.Image 的计算不遵循 Python 的运算符优先级规则，只看操作符的顺序
img = ee.Image(1).add(ee.Image(2)).multiply(ee.Image(3))    # 输出结果为 9，并非先乘后加
img

In [11]:
img = ee.Image(1).add(ee.Image(2).multiply(ee.Image(3)))  # 输出结果为 7，先乘后加，因为先执行了括号中的内容
img

## geemap 计算 ndvi 的方式
### ndvi 是一种用于评估植被生长状况的指标，值域为[-1, 1]
### ndvi 越接近 1，说明植被生长状况越好，小于 0 的部分为非植被
### **ndvi = (NIR - RED) / (NIR + RED)**

In [12]:
# 方法一：使用 ee.Image 自带的方法
ndvi = sentinel.normalizedDifference(['B8', 'B4']).rename('ndvi')

In [13]:
# 方法二：使用 ee.Image.expression() 方法，该方法适用于复杂的计算
ndvi = sentinel.expression(expression='(NIR - RED) / (NIR + RED)',
                            opt_map={'NIR': sentinel.select('B8'), 'RED': sentinel.select('B4')})


In [14]:
# 方法三：使用 ee.Image.select() 方法，先选择波段再计算，该方法适用于简单的计算
NIR = sentinel.select('B8')
RED = sentinel.select('B4')
ndvi = NIR.subtract(RED).divide(NIR.add(RED)).rename('ndvi')

In [15]:
# 方法四：使用开源库 spyndex 计算，该方法适用于各种植被指数的计算，计算需求大时
import spyndex
if 'NDVI' in spyndex.indices:
    print(spyndex.indices["NDVI"])
else:
    print('NDVI 的计算方式 spyndex 库中不存在')


NDVI: Normalized Difference Vegetation Index
        * Application Domain: vegetation
        * Bands/Parameters: ['N', 'R']
        * Formula: (N-R)/(N+R)
        * Reference: https://ntrs.nasa.gov/citations/19740022614
        


In [16]:
ndvi = spyndex.computeIndex(index=['NDVI'],params={'N': sentinel.select('B8'),'R': sentinel.select('B4')}).rename('ndvi')

In [17]:
Map.addLayer(ndvi, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDVI')
Map

Map(bottom=56141.0, center=[25.71826065792773, 115.93872070312501], controls=(WidgetControl(options=['position…

# ee.Image 的**条件运算**

## ee.Image 有诸多条件运算符，常用的有：
### ee.Image.eq() 返回影像中与参数**相等**的 **掩膜（0,1）** 值，相等为 1，不相等为 0，参数可以是**数字**或者**影像**
### ee.Image.neq() 返回影像中**不等于**参数的部分
### ee.Image.gt() 返回影像中**大于**参数的部分
### ee.Image.glt() 返回影像中**大于等于**参数的部分
### ee.Image.lt() 返回影像中**小于**参数的部分
### ee.Image.lte() 返回影像中**小于等于**参数的部分

In [ ]:
# 以 NDVI 为例，NDVI 表达的是植被的茂盛程度，其取值范围在 [-1, 1] 之间，越接近 1 代表植被越茂盛
# 因此为了更好的突出植被，通常将小于 0 的部分赋值为 0
